# Agentic AI with a decision process to select a 'researcher' or 'journalist' as agents using crewAI 

# Instructions:

- Go to the **Assessments** tab in iCollege and click on **Assignments** to find the **Homework 2** assignment and further instructions to the assignment.

- Make a copy of this homework notebook and rename the copy using your GSU username as suffix:

    **AgenticAI_RAG_Homework2_<YOUR GSU USERNAME>.ipynb**

- Use the **MSA8700 Python 3.12** kernel when you execute your noterbnook cells! This kernel has all the necessary Python packages already installed. You do not have to install any packages. Do **NOT** use any of the other kernels. 

- Work with your renamed copy to create your solution and all the necessary .txt files that will be generated and submitted with your notebook. 

- Complete all cells in the notebook i.e. replace missing fields (abbreviated with **'...'** in the code cells below) with your code cells, run the notebook and create the 6 necessary .txt files. 

- Run the tests to confirm that your .txt files were generated. Finally, submit your .txt files and your executed .ipynb notebook by running the two submission steps at the very end of the notebook.

# Important Note:

- When running the 'researcher' branch you might notice that the **`pdf_search_tool` might fail**, i.e. a response cannot be generated fron the PDF file and you get an error message.

- When that occurs **delete the database folder db/** (e.g. with **rm -rf db** on the command line in a terminal window; you might have to restart or shutdown the kernel before you can do that) that has been generated and re-run the cells. The database folder db/ will be re-created and `pdf_search_tool` should return a result with showing an error.

In [ ]:
# Do not change the content of this cell. Execute this cell first, and everytime after you restarted the kernel.
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
from pprint import pprint
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from crewai_tools import PDFSearchTool

### Load the API keys

In [ ]:
# Load your API keys here and create your environment variables.

# IMPORTANT: Do not directly copy your API keys into your notebooks and submit them with your solution!

OPENAI_API_KEY = ...
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

GEMINI_API_KEY = ...
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

SERPER_API_KEY = ...
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

### Load the PDF document

Upload NIPS 2017 paper and store locally in **data/** folder.

Source: 

**Advances in Neural Information Processing Systems 30 (NIPS 2017)**. Edited by: I. Guyon and U. Von Luxburg and S. Bengio and H. Wallach and R. Fergus and S. Vishwanathan and R. Garnett. ISBN: 9781510860964. https://proceedings.neurips.cc/paper_files/paper/2017

**Attention is All you Need** Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez, Łukasz Kaiser, Illia Polosukhin. https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf

This file is already available on the cluster at `/data/MSA8700/data/NIPS-2017-attention-is-all-you-need-Paper.pdf`

In [ ]:
# Check if you can see the PDF file:
! ls -l /data/MSA8700/data/

PDF_FILE = "/data/MSA8700/data/NIPS-2017-attention-is-all-you-need-Paper.pdf"

In [ ]:
import requests

pdf_url = 'https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf'
response = requests.get(pdf_url)

with open('data/NIPS-2017-attention-is-all-you-need-Paper.pdf', 'wb') as file:
    file.write(response.content)

### Initialize LLMs

In [ ]:
# Pick a Gemini mode, the Gemini API key, the max_tokens and temparature parameter values:

llm = ChatOpenAI(api_key=OPENAI_API_KEY)

crew_llm = LLM(
    model = ... ,
    api_key = ... ,
    max_tokens = ... ,
    temperature = ...
)

### Define Tools

In [ ]:
# Initialize the `PDFSearchTool()` class from crewai_tools. Insert the necessary parameters.

# Note: Pick the same model for the language model (llm) and for the embedding model (embedder). Otherwise using  
# the `pdf_search_tool` within an agent task can lead to an error upon execution. 

pdf_search_tool = PDFSearchTool(
    pdf = ...,
    config=dict(
        llm=dict(
            provider = ..., # "openai", "google", "anthropic", "llama2", ...
            config=dict(
                model = ...,
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider = ..., # "openai", "google", "anthropic", "llama2", ...
            config=dict(
                model = ...,
                # task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

In [ ]:
# custom-defined tool using the @tool decrorator:
@tool("Router Function")
def router_tool(question: str) -> str:
  """Router Function"""
  if 'self-attention' in question:
    return 'researcher'
  else:
    return 'journalist'

### Create decision router

In [ ]:
# Create a decision router function based on an input 'query':

# The `decide_hiring()` function shall make a binary decision and return a string. 
# This function shall either return the value 'researcher' or the value 'journalist' 
# depending on whether a certain query can be answered with the content in a PDF file 
# or has to be answered via a web search with web scraping task.

# An agent named 'router' shall execute a task using the tool `pdf_search_tool` to determine whether
# the query can be answered with the PDF file content or has to be answered via a web search with web scraping activity.

# Enter definitions for the variables 'role', 'goal' and 'backstory' of the Agent 'router'.
# Enter definitions for the variables 'description', 'expected_output', 'agent', and 'tools' of the Task 'router_task'.
# Enter definitions for the variables 'agents' and 'tasks' in the Crew class 'crew'. 
# Finally, define the 'inputs' dictionary in order to execute `result = crew.kickoff(inputs={...})` to generate an output string 'result.raw'

# Note: You might have to experiment with different definitions an experiments to get a well-performing agentic workflow.

def decide_hiring(query):
    """Router function to determine if we can answer from the existing PDF file"""    
    
    router = Agent(
                role = ...,
                goal = ...,
                backstory=( ... ),
                verbose=True,
                allow_delegation=False,
                llm=llm,
                )
    
    router_task = Task(
                    description = ( ... ),
                    expected_output = ( ... ),
                    agent = ...,
                    tools = [...],
                    )
    
    crew = Crew(
            agents=[...],
            tasks=[...],
            verbose=1,
            memory=False,
            )
    
    result = crew.kickoff(inputs={...})
    return result.raw

### Create branch workflows with agents, tasks and crews

#### Create the 'researcher' branch in the process: 

In [ ]:
# For the 'researcher' branch, we want to define a function `hire_researcher(query)` that is executed 
# if the 'router' agent determines that the 'query' can be answered by the content avaialble in the PDF file. 

# Define an Agent 'researcher', Task 'researcher_task' and a Crew that will use the tool `pdf_search_tool` to
# provide a response to the query from the PDF tile.

# Similar to the 'router' workflow above:
# Enter definitions for the variables 'role', 'goal' and 'backstory' of the Agent 'researcher'. 
# For the 'role' you can specify 'Expert Data Analyst'.

# Enter definitions for the variables 'description', 'expected_output', 'agent', and 'tools' of the Task 'researcher_task'.
# In the 'description' argument refer to the '{topic}' variable as placeholder for the input query.  

# Enter definitions for the variables 'agents' and 'tasks' in the Crew class 'crew'. 

# By executing `result = crew.kickoff(inputs={"topic": query})` the response from searching the PDF file 
# is generated for the input 'query' by the agentic workflow. 

# Adding the PDF Searching Agent
def hire_researcher(query):     
    # researcher agent:
    researcher = Agent(
                    role = "Expert Data Analyst",
                    goal = ( ... ),
                    backstory = ( ... ),
                    allow_delegation = False,
                    verbose = True,
                    llm = llm
                    )
    
    researcher_task = Task(
                        description = ( ... '{topic}' ... ),
                        expected_output = ( ... ),
                        tools = [ ... ],
                        agent = researcher
                        )
    
    crew = Crew(
            agents = [...],
            tasks = [...],
            verbose=1,
            memory=False,
            ) 
    
    print("The researcher was hired ...")
    result = crew.kickoff(inputs={"topic": query})
    return result.raw

#### Create the 'journalist' branch in the process: 

In [ ]:
# For the 'journalist' branch, alternatively, a function `hire_journalist(query)` is executed 
# if the 'router' agent determines that the 'query' cannot be answered by the content available in the PDF file 
# but needs to be answered via a web searching and web scraping activity. 

# See the Agent definitions of 'web_search_agent' and 'web_scraper_agent'.

# Enter definitions for the variables 'description', 'expected_output', 'agent', and 'tools' 
# of the 'search_task' and 'scraping_task'.

# In the 'description' argument refer to the '{topic}' variable as placeholder for the input query.  

# Enter definitions for the variables 'agents' and 'tasks' in the Crew class 'crew'. 

# By executing `result = crew.kickoff(inputs={"topic": query})` the response 
# from searching and scraping the web.

# Note: Focus on the content scraped from the most relevant searched website with respect to the inout 'query' 
# in order to limit the number of iterations in the web searching and web scraping process. 

# Adding the Web Searching and Scraping Agent
def run_web_search_scraping(query):
    """Setup the web scraping agent and related components"""
    search_tool = SerperDevTool()  # Tool for performing web searches
    scrape_website = ScrapeWebsiteTool()  # Tool for extracting data from websites
    
    # Define the web search agent
    web_search_agent = Agent(
        role="Expert Web Search Agent",
        goal="Identify and retrieve relevant web data for user queries",
        backstory="An expert in identifying valuable web sources for the user's needs",
        allow_delegation=False,
        verbose=True,
        llm=crew_llm
    )
    
    # Define the web scraping agent
    web_scraper_agent = Agent(
        role="Expert Web Scraper Agent",
        goal="Extract and analyze content from specific web page identified by the search agent",
        backstory="A highly skilled web scraper, capable of analyzing and summarizing website content accurately",
        allow_delegation=False,
        verbose=True,
        llm=crew_llm
    )
    
    # Define the web search task
    search_task = Task(
                    description = ( ... '{topic}' ... ),
                    expected_output = ( ... '{topic}' ... ),
                    tools = [...],
                    agent = ...,
                    )
    
    # Define the web scraping task
    scraping_task = Task(
                        description = ( ... '{topic}' ... ),
                        expected_output = ( ... '{topic}' ... ),
                        tools = [...],
                        agent = ...,
                        )
    
    # Define the crew to manage agents and tasks
    crew = Crew(
            agents = [...],
            tasks = [...],
            verbose=1,
            memory=False,
            )
    
    print("The journalist was hired ...")
    result = crew.kickoff(inputs={"topic": query})
    return result.raw


# Complete the 'messages' template for the RAG application in the function `generate_journalist_answer`, 
# i.e. replace the dots in " ... " with an appropriate, short description of a journalist's activity.

def generate_journalist_answer(context, query):
    """Generate final answer using LLM for RAG"""
    messages = [
        (
            "system",
            " ... ",
        ),
        ("system", f"Context: {context}"),
        ("human", query),
    ]
    response = llm.invoke(messages)
    return response.content

In [ ]:
def hire_journalist(query): 
    # Retrieve context from web scraping
    context = run_web_search_scraping(query)
    # Generate final answer via RAG
    answer = generate_journalist_answer(context, query)
    return answer

#### Define process

In [ ]:
# Create the necessary process flow with the functions `decide_hiring`, `hire_researcher`, `hire_journalist` 
# defined above to create the agentic workflow as described above.

# Replace the dots below (...) with your code insertions:

def process_query(query):
    """Main function to process user query"""
    print(f"Processing query: {query}")
    
    # Step 1: Decide which team will be hired based on the question (input query).
    ... 
    
    # Step 2: Generate final answer based on whether 'researcher' or 'journalist' branch was selected.
    
    if ...:
    # researcher branch
        answer = ...
    
    else:
    # journalist branch         
        answer = ... 
            
    return answer

#### Execute process with query

In [ ]:
# Create three different appropriate questions (input query) to test the execution of the 'researcher' branch
# where the 'researcher' agent extracts an answer for the question from the uploaded PDF file:

# Example: "How did the self-attention mechanism evolve in large language models?"

# Choose a different question as input!

In [ ]:
# Pick your first 'question' (replace the ... and insert your string) 
# and execute the `process_query(question)`:
question = " ... "
result = process_query(question)

print("\nFinal Answer:")
pprint(result)

In [ ]:
with open("researcher_1.txt", "w") as text_file:
    text_file.write(result)

In [ ]:
# Pick your second 'question' (replace the ... and insert your string) 
# and execute the `process_query(question)`:
question = " ... "
result = process_query(question)

print("\nFinal Answer:")
pprint(result)

In [ ]:
with open("researcher_2.txt", "w") as text_file:
    text_file.write(result)

In [ ]:
# Pick your third 'question' (replace the ... and insert your string) 
# and execute the `process_query(question)`:
question = " ... "
result = process_query(question)

print("\nFinal Answer:")
pprint(result)

In [ ]:
with open("researcher_3.txt", "w") as text_file:
    text_file.write(result)

In [ ]:
# Pick three different appropriate questions (input query) to test the execution of the 'journalist' branch
# where the 'journalist' agent extracts an answer for the question 
# via a web search and scraping of the most relavant web page found:

# Example: "How does LORA work in large language model fine-tuning?"

# Choose a different question as input!

In [ ]:
# Pick your first 'question' (replace the ... and insert your string) 
# and execute the `process_query(question)`:
question = " ... "
result = process_query(question)

print("\nFinal Answer:")
pprint(result)

In [ ]:
with open("journalist_1.txt", "w") as text_file:
    text_file.write(result)

In [ ]:
# Pick your second 'question' (replace the ... and insert your string) 
# and execute the `process_query(question)`:
question = " ... "
result = process_query(question)

print("\nFinal Answer:")
pprint(result)

In [ ]:
with open("journalist_2.txt", "w") as text_file:
    text_file.write(result)

In [ ]:
# Pick your third 'question' (replace the ... and insert your string) 
# and execute the `process_query(question)`:
question = " ... "
result = process_query(question)

print("\nFinal Answer:")
pprint(result)

In [ ]:
with open("journalist_3.txt", "w") as text_file:
    text_file.write(result)

# Submission Instructions:

- Submit your **6 responses** (three reponses each for the two branches listed above) as text files described above.

- Submit your **executed (!) Jupyter notebook**. See instructions below.

- **Important:** Your notebook solution needs to be executable with the Python packages listed and return correct answers with the selected queries.

# Homework submission on new IFI cluster:
- This homework is due by **2025-03-31, 2:30 PM (EDT) (Section 006)** or **2025-03-31, 6:00 PM (EDT) (Section 003)**, respectively.
- Make sure that all your programs and output files are in the exact folder as specified in the instructions.
- All file names on this system are case sensitive. Verify if you copy your work from a local computer to your home directory on the new cluster.
- **Execute the cell below to submit your assignment (6 text files and executed notebook):**

In [ ]:
# Execute this cell. DO NOT EDIT THIS CONTENT!
from msa8700 import Homework2
Homework2(__session__).submit().list()